# preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
'''load data'''
def loaddata(path):
    df= pd.read_excel(path,header=0)
    df= df.rename(columns={'C（wt.%)':'C','T(℃)':'T','σγ(MPa)':'sigma','ΔGγ→α(J)':'dG','Wαβ(μm)':'W'})
    df= df.reset_index(drop= True) # drop index 
    
    return df

'''调整数据'''
def null_filled(df):
    df= df.loc[:,'T':'W']
    df.W= df.W*1000
    df= df.apply(pd.to_numeric,errors='coerce')# pd.to_numeric transform to numeric
    return df

'''preprocessing'''
def scale(df):
    array= df.as_matrix() # as_matrix 

    scaler= StandardScaler()
    scaler.fit(array)
    array= scaler.transform(array)
    return array
    

df1= loaddata(r'E:\文档\贝氏体板条宽度W公式拟合.xlsx')
df2= null_filled(df1)

data= scale(df2)

from sklearn.linear_model import LinearRegression
LiRe= LinearRegression()
LiRe.fit(data[:,0:3],data[:,3])
print(LiRe.score(data[:,0:3],data[:,3]))
print(LiRe.coef_,'\n',LiRe.intercept_)


0.807817878729
[ 0.56507209 -0.44084841 -0.06443341] 
 3.53172043432e-16


# NN

In [4]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.autograd import Variable

In [5]:

'''data '''
x_train,x_test,y_train,y_test= train_test_split(data[:,0:3],data[:,3],test_size=0.3)
x= torch.from_numpy(x_train).type(torch.FloatTensor)
y= torch.unsqueeze(torch.from_numpy(y_train).type(torch.FloatTensor),dim=1)

x_test= torch.from_numpy(x_test).type(torch.FloatTensor)
y_test= torch.unsqueeze(torch.from_numpy(y_test).type(torch.FloatTensor),dim=1)
'''variablize'''
x= Variable(x)
y= Variable(y)

'''net'''
class Net(torch.nn.Module):
    def __init__(self,n_h1,n_h2):
        super().__init__()
        self.linear1= nn.Sequential(
                    nn.Linear(3,n_h1),
                    nn.Tanh())
        self.linear2= nn.Sequential(
                    nn.Linear(n_h1,n_h2),
                    nn.Tanh())
        self.predict= nn.Linear(n_h2,1)

    def forward (self,x):
        x= self.linear1(x)
        x= self.linear2(x)
        x= self.predict(x)
        return x

net1= Net(16,64)
# print(net1)

'''optimizer'''
optimizer= torch.optim.Adam(net1.parameters(),lr=0.1)
'''loss funciton'''
loss_func= torch.nn.MSELoss()

'''iteration'''
for k in range(400):
    prediction= net1(x)
    loss= loss_func(prediction,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if k%50 ==0:
        print(loss.data[0])

'''test'''
y2= net1(Variable(x_test))

MSE= F.mse_loss(y2.data,y_test)
print(MSE)

'''get parameters'''
#     for name,param in net1.named_parameters():
#         if param.requires_grad:
#             print(name,param.data)

#     for param in net1.parameters():
#         print(param)
'''save model'''    
torch.save(net1.state_dict(),r'E:\文档\net1.pkl')




0.9404581189155579
0.13027901947498322
0.23567970097064972
0.03926798328757286
0.031896211206912994
0.03538757190108299
0.03554120287299156
0.026335865259170532
Variable containing:
 0.3555
[torch.FloatTensor of size 1]



# load model

In [6]:
net2= Net(16,64)
net2.load_state_dict(torch.load(r'E:\文档\net1.pkl'))
y2= net2(Variable(x_test))
print(F.mse_loss(y2.data,y_test))

Variable containing:
 0.3555
[torch.FloatTensor of size 1]

